Make sure you download the 2016 Household LSMS survey data for Malawi, Ethiopia, and Nigeria from https://microdata.worldbank.org/index.php/catalog/lsms and put it in `../data/countries/`. Malawi's data should be named `nigeria_2019/LSMS`, Ethiopia's should be named `nigeria_2019/LSMS`, and Nigeria's should be named `nigeria_2019/LSMS`. Nightlights data should be downloaded from https://ngdc.noaa.gov/eog/viirs/download_dnb_composites.html using the annual composite from 2015 in tile 2 and tile 5.

In [1]:
import pandas as pd
import numpy as np
import os
import geoio

In [2]:
BASE_DIR = '..'
NIGHTLIGHTS_DIRS = [os.path.join(BASE_DIR, 'data/nightlights/avg_nightlights_2019.tif')]
COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')

In [3]:
import sys
sys.path.append(BASE_DIR)
from utils import create_space

In [6]:
'''
The goal of each of these functions is to output a dataframe with the following columns:
country, cluster_lat, cluster_lon, cons_pc

Each row should represent one cluster by combining the household data
'''

def process_malawi():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'malawi_2019', 'LSMS')
    consumption_file = 'ihs5_consumption_aggregate.csv'
    consumption_ph_col = 'rexpagg' # per household
    hhsize_col = 'hhsize' # people in household

    geovariables_file = 'householdgeovariables_ihs5.csv'
    lat_col = 'ea_lat_mod'
    lon_col = 'ea_lon_mod'

    # purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
    ppp = 265.2
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_ph_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['case_id', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['case_id', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='case_id')
    df_combined.drop(['case_id'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'mw'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]
    
def process_ethiopia():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'ethiopia_2019', 'LSMS')
    consumption_file = 'cons_agg_w4.csv'
    consumption_pc_col = 'total_cons_ann' # per capita
    hhsize_col = 'hh_size' # people in household

    geovariables_file = 'ETH_HouseholdGeovariables_Y4.csv'
    lat_col = 'lat_mod'
    lon_col = 'lon_mod'

    # purchasing power parity for ethiopia in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=ET)
    ppp = 10.74
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['household_id', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['household_id', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='household_id')
    df_combined.drop(['household_id'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'eth'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

def process_nigeria():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'nigeria_2019', 'LSMS')
    consumption_file = 'totcons_final.csv'
    consumption_pc_col = 'totcons_pc' # per capita
    hhsize_col = 'hhsize' # people in household

    geovariables_file = 'nga_householdgeovars_y4.csv'
    lat_col = 'lat_dd_mod'
    lon_col = 'lon_dd_mod'

    # purchasing power parity for nigeria in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=NG)
    ppp = 134.21
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['hhid', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['hhid', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='hhid')
    df_combined.drop(['hhid'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'ng'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

In [7]:
df_mw = process_malawi()

C:\Users\Dima\AppData\Local\Temp\ipykernel_13532\869959384.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)


In [8]:
df_eth = process_ethiopia()

C:\Users\Dima\AppData\Local\Temp\ipykernel_13532\869959384.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)


In [9]:
df_ng = process_nigeria()

C:\Users\Dima\AppData\Local\Temp\ipykernel_13532\869959384.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)


In [10]:
df_mw.shape, df_eth.shape, df_ng.shape

((709, 4), (516, 4), (642, 4))

In [11]:
tifs = [geoio.GeoImage(ndir) for ndir in NIGHTLIGHTS_DIRS]

In [78]:
# loading both of these into memory requires A LOT of free memory (at least 4 gigs)
# using a swapfile of size 2 GB still did not fix my issues
# instead, I knew ahead of time the 0th tif is for Malawi, and the 1st tif is for Ethiopia and Nigeria
# I'll use this to only load one tif at a time
# thankfully, the countries did not span across two tifs
tif_array = np.squeeze(tifs[0].get_data())

In [79]:
def add_nightlights(df, tif, tif_array):
    ''' 
    This takes a dataframe with columns cluster_lat, cluster_lon and finds the average 
    nightlights in 2015 using a 10kmx10km box around the point
    
    I try all the nighlights tifs until a match is found, or none are left upon which an error is raised
    '''
    cluster_nightlights = []
    for i,r in df.iterrows():
        min_lat, min_lon, max_lat, max_lon = create_space(r.cluster_lat, r.cluster_lon)
        
        xminPixel, ymaxPixel = tif.proj_to_raster(min_lon, min_lat)
        xmaxPixel, yminPixel = tif.proj_to_raster(max_lon, max_lat)
        assert xminPixel < xmaxPixel, print(r.cluster_lat, r.cluster_lon)
        assert yminPixel < ymaxPixel, print(r.cluster_lat, r.cluster_lon)
        if xminPixel < 0 or xmaxPixel >= tif_array.shape[1]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        elif yminPixel < 0 or ymaxPixel >= tif_array.shape[0]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
        cluster_nightlights.append(tif_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())
        
    df['nightlights'] = cluster_nightlights

In [80]:
add_nightlights(df_eth, tifs[0], tif_array)

In [83]:
import matplotlib.pyplot as plt

# Plotting the Data
plt.imshow(tif_array, cmap='gray')
plt.colorbar()
plt.title("Nightlight Intensity Distribution")
plt.savefig('nightlight_distribution.png')  # Save the plot as an image file
plt.close()  # Close the plot to free up memory

MemoryError: Unable to allocate 10.8 GiB for an array with shape (33601, 86401) and data type float32

In [50]:
add_nightlights(df_mw, tifs[0], tif_array)

In [55]:
add_nightlights(df_ng, tifs[0], tif_array)

In [84]:
del tif_array
import gc
gc.collect()

948

In [85]:
import psutil
psutil.virtual_memory()

svmem(total=14835068928, available=350760960, percent=97.6, used=14484307968, free=350760960)

In [58]:
#next 3 cells not needed this time, bc only 1 TIF file instead of 2

In [59]:
#tif_array = np.squeeze(tifs[1].get_data())

In [60]:
#add_nightlights(df_eth, tifs[1], tif_array)

In [61]:
#add_nightlights(df_ng, tifs[1], tif_array)

In [62]:
df_mw.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,mw,-17.093531,35.253139,1.994561,0.034344
1,mw,-17.065680,35.166790,1.834210,0.001391
2,mw,-17.028139,35.241661,1.620246,0.012070
3,mw,-17.005220,35.082470,1.619339,0.000000
4,mw,-16.964531,35.208618,1.519061,0.061038


In [63]:
# There seems to be an erroneous outlier
lat_min = df_mw['cluster_lat'].min()
lat_max = df_mw['cluster_lat'].max()
lon_min = df_mw['cluster_lon'].min()
lon_max = df_mw['cluster_lon'].max()

print(f"Latitude Min: {lat_min}, Max: {lat_max}")
print(f"Longitude Min: {lon_min}, Max: {lon_max}")


Latitude Min: -17.0935306549072, Max: -9.39844036102295
Longitude Min: 32.8477096557617, Max: 35.87451171875


In [52]:
#remove outlier
# Define a threshold near zero to identify outliers
outlier_threshold = 0.1

# Filter out the outliers where both latitude and longitude are within the outlier threshold directly in the df_mw_download DataFrame
df_mw = df_mw[~((abs(df_mw['cluster_lat']) < outlier_threshold) & 
                                  (abs(df_mw['cluster_lon']) < outlier_threshold))]


In [64]:
df_eth.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,eth,3.609384,39.021503,16.641462,0.010387
1,eth,4.010758,41.765862,8.226561,0.000000
2,eth,4.439883,41.876270,18.858865,0.000000
3,eth,4.730678,41.537197,16.841644,0.000000
4,eth,4.744136,36.045395,5.768457,0.000000


In [65]:
# checking for outliers
lat_min = df_eth['cluster_lat'].min()
lat_max = df_eth['cluster_lat'].max()
lon_min = df_eth['cluster_lon'].min()
lon_max = df_eth['cluster_lon'].max()

print(f"Latitude Min: {lat_min}, Max: {lat_max}")
print(f"Longitude Min: {lon_min}, Max: {lon_max}")


Latitude Min: 3.60938429832459, Max: 14.4771537780762
Longitude Min: 33.4348297119141, Max: 47.3078384399414


In [24]:
df_ng.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,ng,4.328719,6.308172,4.595377,0.126053
1,ng,4.425298,5.844956,5.861942,0.501679
2,ng,4.492056,7.432757,4.764470,22.507589
3,ng,4.532173,7.771942,3.940768,0.000000
4,ng,4.537820,6.400036,4.605447,0.193849


In [67]:
# checking
lat_min = df_ng['cluster_lat'].min()
lat_max = df_ng['cluster_lat'].max()
lon_min = df_ng['cluster_lon'].min()
lon_max = df_ng['cluster_lon'].max()

print(f"Latitude Min: {lat_min}, Max: {lat_max}")
print(f"Longitude Min: {lon_min}, Max: {lon_max}")


Latitude Min: 4.32871914124, Max: 13.7142474624
Longitude Min: 2.814713074, Max: 13.5108863216


In [68]:
df_mw['nightlights'].mean()

0.5075015

In [86]:
df_eth['nightlights'].mean()

1.3873873

In [70]:
df_ng['nightlights'].mean()

1.6184558

In [71]:
df_mw.select_dtypes(include=[np.number]).corr() #correlation

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,-0.711741,0.112722,-0.048643
cluster_lon,-0.711741,1.000000,-0.023748,-0.057093
cons_pc,0.112722,-0.023748,1.000000,0.583380
nightlights,-0.048643,-0.057093,0.583380,1.000000


In [72]:
df_eth.select_dtypes(include=[np.number]).corr() #correlation

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.015297,-0.079680,-0.058496
cluster_lon,0.015297,1.000000,0.195609,0.038725
cons_pc,-0.079680,0.195609,1.000000,0.422126
nightlights,-0.058496,0.038725,0.422126,1.000000


In [73]:
df_ng.select_dtypes(include=[np.number]).corr() #correlation

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.338771,-0.409694,-0.169368
cluster_lon,0.338771,1.000000,-0.315270,-0.299754
cons_pc,-0.409694,-0.315270,1.000000,0.384976
nightlights,-0.169368,-0.299754,0.384976,1.000000


In [74]:
for country in ['malawi_2019', 'ethiopia_2019', 'nigeria_2019']:
    os.makedirs(os.path.join(COUNTRIES_DIR, country, 'processed'), exist_ok=True)

In [75]:
df_mw.to_csv(os.path.join(COUNTRIES_DIR, 'malawi_2019', 'processed/clusters.csv'), index=False)

In [76]:
df_eth.to_csv(os.path.join(COUNTRIES_DIR, 'ethiopia_2019', 'processed/clusters.csv'), index=False)

In [77]:
df_ng.to_csv(os.path.join(COUNTRIES_DIR, 'nigeria_2019', 'processed/clusters.csv'), index=False)